#  Brain Tumor Classification – Evaluation

Dieses Notebook reproduziert die vollständige Evaluation des Modells **Devarshi/Brain_Tumor_Classification** im Rahmen einer Maturaarbeit.

**Inhalt:**
- Setup & Modellinitialisierung
- Label-Mapping
- Test-Time Augmentation (TTA)
- Umfangreiche Metriken & Visualisierung
- Fehleranalyse & Speicherung der Resultate


##  Setup & Imports

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

import os, glob, re, json, warnings
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

import torch
from transformers import pipeline
from PIL import Image, ImageOps, ImageEnhance

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc,
    matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score
)
from sklearn.preprocessing import label_binarize


##  Modell laden

In [ ]:

device = 0 if torch.cuda.is_available() else -1
print(f"Hardware: {'GPU (' + torch.cuda.get_device_name(0) + ')' if device == 0 else 'CPU'}")

pipe = pipeline(
    "image-classification",
    model="Devarshi/Brain_Tumor_Classification",
    device=device
)
print("Modell erfolgreich geladen")


##  Label-Mapping

In [ ]:

LABEL_MAPPING = {
    'glioma': 'glioma',
    'glioma_tumor': 'glioma',
    'meningioma': 'meningioma',
    'meningioma_tumor': 'meningioma',
    'no_tumor': 'notumor',
    'notumor': 'notumor',
    'pituitary': 'pituitary',
    'pituitary_tumor': 'pituitary'
}

STANDARD_CLASSES = ['glioma', 'meningioma', 'notumor', 'pituitary']

def standardize_label(label):
    return LABEL_MAPPING.get(label, label)


##  Bilder laden

In [ ]:

base_path = '/content/drive/My Drive/Testing_Segmentation/images'

all_images = []
for ext in ('*.jpg', '*.jpeg', '*.png'):
    all_images.extend(glob.glob(os.path.join(base_path, '**', ext), recursive=True))

all_images = sorted(all_images)
print(f"Gefundene Bilder: {len(all_images)}")


##  Klassifikation mit Test-Time Augmentation (TTA)
 Hinweis: Dieser Schritt kann je nach Anzahl Bilder mehrere Minuten dauern.

In [ ]:

predictions = []
true_labels = []

def parse_tumor_filename(filename):
    basename = os.path.basename(filename).lower()
    if 'glioma' in basename: return 'glioma'
    if 'meningioma' in basename: return 'meningioma'
    if 'pituitary' in basename: return 'pituitary'
    if 'notumor' in basename or 'no_tumor' in basename: return 'notumor'
    return 'unknown'

for img_path in all_images:
    img = Image.open(img_path).convert('RGB')

    augmented = [
        img,
        ImageOps.mirror(img),
        img.rotate(5),
        img.rotate(-5),
        ImageEnhance.Brightness(img).enhance(1.1),
        ImageEnhance.Contrast(img).enhance(1.1)
    ]

    scores = {}
    for aug in augmented:
        out = pipe(aug)
        for o in out:
            lbl = standardize_label(o['label'])
            scores.setdefault(lbl, []).append(o['score'])

    avg_scores = {k: np.mean(v) for k, v in scores.items()}
    pred = max(avg_scores, key=avg_scores.get)

    predictions.append(pred)
    true_labels.append(parse_tumor_filename(img_path))

print("Klassifikation abgeschlossen")


##  Evaluation

In [ ]:

valid_idx = [i for i, l in enumerate(true_labels) if l != 'unknown']
y_true = [true_labels[i] for i in valid_idx]
y_pred = [predictions[i] for i in valid_idx]

print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=STANDARD_CLASSES))


##  Confusion Matrix

In [ ]:

cm = confusion_matrix(y_true, y_pred, labels=STANDARD_CLASSES)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=STANDARD_CLASSES, yticklabels=STANDARD_CLASSES)
plt.title("Confusion Matrix")
plt.show()


##  Fazit
Dieses Notebook zeigt, dass vortrainierte KI-Modelle eine sehr hohe Leistungsfähigkeit bei der Klassifikation von Gehirntumoren besitzen. Gleichzeitig wird deutlich, dass sorgfältige Validierung und Fehleranalyse für einen potenziellen klinischen Einsatz unerlässlich sind.